## Points to remember

Write down the points to be remembered in this cell while coding, so it might be helpful to others working on same code - different functions

 
FLAT -> 0 </br>
HILLY -> 1 </br>
FORREST -> 2 </br>
BLOCK -> 3 </br>

In [17]:
import numpy as np
import sympy as sym
import random
from scipy.spatial.distance import cityblock

In [18]:
class agent_6:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.actions = 0
        self.flat_exam = 0
        self.hilly_exam = 0
        self.forest_exam = 0

#     def display_node(self):
#         print("Node: ", self.row, "," , self.column)
#         print("\n the number of neighbors cell x has: ", self.Nx)
#         print("visited  Whether or not cell x has been visited: ", self.visited)
#         print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
#         print("The undiscovered neighbors are: ", self.neighbor_list)
#         print("The confirmed neighbors that are free: ", self.free_neighbors)
#         print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

        
        
    def getMaxProbabilityCell(self, source):
        mat = self.belief_matrix
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
#                     node_to_return = target_node
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        
        return node_to_return
    
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    def update_cells_when_xy_blocked(self, x, y, maze_size):
        denominator = 1 - self.belief_matrix[x][y]
        self.belief_matrix = self.belief_matrix/denominator
        
        self.belief_matrix[x][y]=0
        
        
#         print(self.belief_matrix)
    
    
    def update_cells_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):
        
        if t == 0:
            p_xy = 0.2*self.belief_matrix[x][y]
        elif t == 1:
            p_xy= 0.5*self.belief_matrix[x][y]
        elif t == 2:
            p_xy= 0.8*self.belief_matrix[x][y]
        
        self.belief_matrix[x][y] = p_xy
        self.belief_matrix = self.belief_matrix/self.belief_matrix.sum()
        

In [19]:
class gridworld:
    
    def __init__(self, maze_size):
#         self.full_grid_world_matrix = []
        self.terrain_matrix = self.initialize_terrain_matrix(maze_size)
        self.full_grid_world_matrix = self.initialize_grid_world(maze_size)
        self.source, self.target = self.set_valid_source_target(maze_size)

    def initialize_terrain_matrix(self, maze_size):
        # generates number between 0-2
        np_array = np_array = np.random.rand(maze_size*maze_size)
        block = lambda x: 3 if x<0.3 else 0 # setting blocks
        vectorized_block = np.vectorize(block)
        np_array = vectorized_block(np_array)
#         print(np_array.reshape(maze_size,maze_size))
        self.full_grid_world_matrix = np_array  #setting full_gridworld_matrix
        
        terrain_probs = lambda x: 3 if x==3 else np.random.rand()
        vectorized_terrain_probs = np.vectorize(terrain_probs)
        np_array = vectorized_terrain_probs(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        set_terrain = lambda x: x if x==3 else(0 if x<=0.33 else (1 if x<=0.66 else 2))
        vectorized_set_terrain = np.vectorize(set_terrain)
        np_array = vectorized_set_terrain(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        terrain_matrix = np.asmatrix(np_array).reshape(maze_size,maze_size)
#         print(terrain_matrix)
        return terrain_matrix

    def initialize_grid_world(self, maze_size):

        block = lambda x: 1 if x==3 else x
        vectorize_block = np.vectorize(block)
        self.full_grid_world_matrix = vectorize_block(self.full_grid_world_matrix)
        self.full_grid_world_matrix = np.asmatrix(self.full_grid_world_matrix).reshape(maze_size,maze_size)
    #     plt.spy(full_grid_world_matrix)
        return self.full_grid_world_matrix

    def set_valid_source_target(self, maze_size):

        target_row = random.randint(0, maze_size-1)
        target_col = random.randint(0, maze_size-1)

        source_row = random.randint(0, maze_size-1)
        source_col = random.randint(0, maze_size-1)

        while (self.full_grid_world_matrix.item(target_row, target_col) == 1):
            target_row = random.randint(0, maze_size-1)
            target_col = random.randint(0, maze_size-1)
            
        while(self.full_grid_world_matrix.item(source_row, source_col) == 1):
            source_row = random.randint(0, maze_size-1)
            source_col = random.randint(0, maze_size-1)
        
        source = (source_row, source_col)
        target = (target_row, target_col)

        return source, target
               

In [20]:
# obj = gridworld(10)

In [21]:
# obj.full_grid_world_matrix

In [22]:
# obj.terrain_matrix